In [ ]:
import os
import json
import openai
from datetime import datetime, timedelta
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage
from simple_salesforce import Salesforce
import pprint

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
completion = openai.ChatCompletion.create(
    model="gpt-4o-2024-05-13",
    messages=[
        {
            "role": "user",
            "content": "What are my deals in salesforce?",
        },
    ],
)
output = completion.choices[0].message.content
print(output)

In [ ]:
username = os.getenv('SALESFORCE_USERNAME')
password = os.getenv('SALESFORCE_PASSWORD')
security_token = os.getenv('SALESFORCE_SECURITY_TOKEN')

sf = Salesforce(username=username, password=password, security_token=security_token)


def execute_sosl(search):
    """Executes a SOSL search with the given search string."""
    return f"SOSL search result for: {search}"

def execute_soql(query):
    """Executes a SOQL query with the given query string."""
    return f"SOQL query result for: {query}"


In [ ]:
function_descriptions = [
    {
            "name": "execute_sosl",
            "description": "Executes a SOSL search with the given search string.",
            "parameters": {
                "type": "object",
                "properties": {
                    "search": {
                        "type": "string",
                        "description": "The SOSL search string to be executed."
                    }
                },
                "required": ["search"]
            },
        },

        {
            "name": "execute_soql",
            "description": "Executes a SOQL query with the given query string.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The SOQL query string to be executed."
                    }
                },
                "required": ["query"]
            },
        }
]

In [ ]:
user_message = "What are my deals in Salesforce?"

response = openai.ChatCompletion.create(
    model="gpt-4o-2024-05-13",
    messages=[
        {"role": "user", "content": user_message}
    ],
    functions=function_descriptions,
    function_call="auto",
)

output = response.choices[0].message

if isinstance(output, dict) and "function_call" in output:
    function_name = output["function_call"]["name"]
    function_args = json.loads(output["function_call"]["arguments"])

    if function_name in globals():
        chosen_function = globals()[function_name]
        function_result = chosen_function(**function_args)

        function_response = f"{function_name} result: {function_result}"

        completion = openai.ChatCompletion.create(
            model="gpt-4o-2024-05-13",
            messages=[
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": output["content"], "function_call": output["function_call"]},
                {"role": "function", "content": function_response, "name": function_name}
            ],
            functions=function_descriptions,
            function_call="auto",
        )

        output = completion.choices[0].message
        print(output)
else:
    if isinstance(output, dict) and "content" in output:
        print(output["content"])
    else:
        print(output)